In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import json
import logging
from marshaltools import ProgramList
from astropy.time import Time
import astropy.units as u
import os

from Observatory import Observatory
from utils import get_config, plot_visibility, prepare_snifs_schedule

In [ ]:
# If you are running this notebook for the first time, you will be prompted to enter your username and
# the path to your downloads from slack. If you entered something incorrectly, you can correct this in .config
#config = get_config()

username = 'jnordin'
date = '2021-02-14'
obs = 'uh88'
tshift = {'ntt': -0.25, 'uh88': 0}  # In units of days
trange = [Time(Time(date).jd+tshift[obs],format='jd'), Time(Time(date).jd+1.+tshift[obs],format='jd')]

logpath = 'logs/'

# log
logger = logging.getLogger(username)
handler = logging.FileHandler(os.path.join(logpath, 'ztfcosmo_visibilty_%s_%s_%s.log'%(obs,date,username)))
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

In [ ]:
observatories ={
    'uh88': Observatory('uh88', 19.8231, -155.47, 4205, sun_alt_th=-17, logger=logger),
    'ntt': Observatory('ntt', -29.25, -70.73, 2375, sun_alt_th=-12, logger=logger), # What's the twilight limit for ePESSTO?
}

In [ ]:
# Calculate dark time and observation windows
dark_time = observatories[obs].get_dark_times(trange)

In [ ]:
# Load one of the marshal programs to get the coordinates of your targets (pick one that has all/most of them)
pl = ProgramList('Redshift Completeness Factor')

In [ ]:
# If you have a commented list of 

targets_raw = """
https://fritz.science/source/ZTF21aakilyd P1
FBOT
https://fritz.science/source/ZTF21aagpbkj P1
https://fritz.science/source/ZTF18aatgdph P1
https://fritz.science/source/ZTF21aagkwjc P1
https://fritz.science/source/ZTF21aagmqic P4
https://fritz.science/source/ZTF21aaffxdt P2
https://fritz.science/source/ZTF21aagkzwn P4
https://fritz.science/source/ZTF21aagogqe P4
https://fritz.science/source/ZTF21aagsypp P2
https://fritz.science/source/ZTF21aagqcnl P3
https://fritz.science/source/ZTF21aagojfe P4

https://fritz.science/source/ZTF21aagmudi P3
https://fritz.science/source/ZTF21aahptas P2
https://fritz.science/source/ZTF21aaixlfe P4
extinction
https://fritz.science/source/ZTF21aahvuvi P3
https://fritz.science/source/ZTF21aakupth P1
https://fritz.science/source/ZTF21aahsgan P2
(fairly early)


https://fritz.science/source/ZTF21aaglret P3"""

In [ ]:
# List your targets here
targets_prio = [a.split('/')[-1] for a in targets_raw.split('\n') if a.startswith('https')]
print(targets_prio)

In [ ]:
import requests
root_token = "enter_token"
base_url = "https://fritz.science"
session = requests.Session()

In [ ]:
# Obtaincoordinates
targets = {}
for t in targets_prio:
    print(t)
    name, prio = t.split(' ')
    print(name)
    reply = session.request( "GET", base_url+"/api/candidates/"+name, headers =  {"Authorization": f"token {root_token}"})
    print(reply)
    print(reply.json()['data']['ra'])
    targets[name] = {'ra':reply.json()['data']['ra'], 'dec': reply.json()['data']['dec'], 'prio':prio}

In [ ]:
targets

In [ ]:
obs_windows = {}

In [ ]:
observatories[obs].min_moon_dist = 30

In [ ]:
for snname in targets.keys():
    ra = targets[snname]['ra']
    dec = targets[snname]['dec']
    logger.info('Calculating visibility of source %s (ra: %f, dec: %f) with %s.'%(snname, ra, dec, obs))
    obs_w = observatories[obs].compute_visibility(ra, dec, trange)
    if len(obs_w['visible']) > 0:
        obs_windows[snname] = obs_w
    else:
        logger.info('Source %s (ra: %f, dec: %f) not observable with %s.'%(snname, ra, dec, obs))

In [ ]:
print('%i of %i targets are observable with %s.'%(len(obs_windows.keys()), len(targets), obs))

In [ ]:
print([n_ for n_ in targets if n_ not in obs_windows.keys()])

In [ ]:
# Sort the SNe by time of lowest airmass etc
snnames = [k for k in obs_windows.keys()]

# Sort by end of visibility
f_sort = lambda a: np.where(dark_time == obs_windows[a]['visible'][-1])[0][0]
snnames = sorted(snnames, key=f_sort)
# Sort by start of visibility
f_sort = lambda a: np.where(dark_time == obs_windows[a]['visible'][0])[0][0]
snnames = sorted(snnames, key=f_sort)
# Sort by time of best airmass
f_sort = lambda a: np.where(dark_time == obs_windows[a]['visible'][obs_windows[a]['airmass'] == min(obs_windows[a]['airmass'])])[0][0]
snnames = sorted(snnames, key=f_sort)

# Priorities need to be assigned (semi)-manually using a dictionary of the target names 
# (easy if they are sorted by prio already) 
sort_dict = {}
for k, n_ in enumerate(targets):
    if targets[n_]['prio']=='P1':
        sort_dict[n_] = 0
    elif targets[n_]['prio']=='P2':
        sort_dict[n_] = 1
    elif targets[n_]['prio']=='P3':
        sort_dict[n_] = 2
    elif targets[n_]['prio']=='P4':
        sort_dict[n_] = 3
    else:
        sort_dict[n_] = 4
    
f_sort = lambda a: sort_dict[a]
snnames = sorted(snnames, key=f_sort)
priorities = [f_sort(a) for a in snnames]

In [ ]:
priorities

In [ ]:
_ = plot_visibility(snnames, obs_windows, dark_time, priorities)

In [ ]:
# Selected the targets by their indices
selected = [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
# Set manual priorities if you wish
prio_new = []
print('%i SNe selected'%len(selected))

snnames_selected = [snnames[k] for k in selected]
priorities_selected = [priorities[k] for k in selected]

sort_again = False
if sort_again:
    # sort them by time of lowest airmass again
    f_sort = lambda a: np.where(dark_time == obs_windows[a]['visible'][obs_windows[a]['airmass'] == min(obs_windows[a]['airmass'])])[0][0]
    if len(prio_new) < len(selected):
        tmp = sorted(zip(snnames_selected, priorities_selected) , key=lambda a: f_sort(a[0]))
    else:
        tmp = sorted(zip(snnames_selected, prio_new) , key=lambda a: f_sort(a[0]))
    snnames_selected = [a[0] for a in tmp]
    priorities_selected = [a[1] for a in tmp]

_ = plot_visibility(snnames_selected, obs_windows, dark_time, priorities_selected, labeltype='priority')
plt.savefig('snifs/targets_snifs_%s.png'%date)

In [ ]:
if obs == 'uh88':
    if len(prio_new) == len(selected):
        snnames_out = [snnames[k] for k, p in sorted(zip(selected, prio_new), key=lambda a: a[1])]
    else:
        snnames_out = [snnames[k] for k in selected]
    prepare_snifs_schedule(snnames_out, targets, obs_windows, date, logger=logger)